8)the winner prediction on 3*3 game. Similar to provided code. Just with values property

In [17]:
from GraphTsetlinMachine.graphs import Graphs
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time
import argparse
from skimage.util import view_as_windows
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib

In [18]:
import os
current_directory = os.getcwd()
print(current_directory)
dir_base = os.path.join(current_directory,'3_3_small.csv')
print(dir_base)

/home/ladans/Learning systems/Sample3_3
/home/ladans/Learning systems/Sample3_3/3_3_small.csv


In [19]:
import pandas as pd

try:
    data = pd.read_csv(dir_base)
    if data.empty:
        print("The CSV file is empty.")
    else:
        print("Data loaded successfully.")
except pd.errors.EmptyDataError:
    print("No data found in the CSV file.")
except FileNotFoundError:
    print("The file was not found. Please check the path.")


Data loaded successfully.


In [20]:
# Define the function to convert the board into numeric values (1 for 'X', 2 for 'O', 0 for space)
def convert_board_to_numeric(board):
    return [1 if ch == 'X' else 2 if ch == 'O' else 0 for ch in board]

# Ensure you are working with df, not data
data['numeric_board'] = data['board'].apply(convert_board_to_numeric)

# Convert each row of numeric_board to a 3x3 matrix
data['numeric_board_matrix'] = data['numeric_board'].apply(lambda x: np.array(x).reshape(3, 3))

# Display the numeric_board_matrix column to see the 3x3 matrices
data[['board', 'numeric_board_matrix', 'winner']].head()

,board,numeric_board_matrix,winner
0,X XXOOO,"[[0, 1, 0], [1, 1, 2], [2, 2, 0]]",1
1,XOOXXXOOX,"[[1, 2, 2], [1, 1, 1], [2, 2, 1]]",0
2,OOXXXXXOO,"[[2, 2, 1], [1, 1, 1], [1, 2, 2]]",0
3,OXXOXXOOX,"[[2, 1, 1], [2, 1, 1], [2, 2, 1]]",0
4,OOOXX XXO,"[[2, 2, 2], [1, 1, 0], [1, 1, 2]]",1


In [21]:
data['numeric_board_matrix'].iloc[0]

array([[0, 1, 0],
       [1, 1, 2],
       [2, 2, 0]])

In [22]:
input_matrix=data['numeric_board_matrix']
print(len(input_matrix))

2000


In [23]:
board_size = 3
subset_size = int(data.shape[0] * 0.9)
test_size = data.shape[0] - subset_size
X_train = input_matrix.iloc[:subset_size]
X_train
X_test = data['numeric_board_matrix'].iloc[subset_size:subset_size + test_size]
Y_train = data['winner'].iloc[:subset_size]
Y_test = data['winner'].iloc[subset_size:subset_size + test_size]

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
X_train[0]

X_train shape: (1800,)
X_test shape: (200,)


array([[0, 1, 0],
       [1, 1, 2],
       [2, 2, 0]])

In [24]:
Y_train

0       1
1       0
2       0
3       0
4       1
       ..
1795    0
1796    1
1797    1
1798    1
1799    0
Name: winner, Length: 1800, dtype: int64

In [25]:
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()
X_train_list = np.array(X_train_list)
X_test_list = np.array(X_test_list)
print(len(X_train_list))
X_train_list.shape

1800


(1800, 3, 3)

In [26]:
from GraphTsetlinMachine.graphs import Graphs
import numpy as np
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time


dim = 3

number_of_nodes = dim * dim


symbol_names = ["Blue", "Red", "White"]

symbol_names

['Blue', 'Red', 'White']

In [27]:
# Initialize the graph object with appropriate symbols and settings
graphs_train = Graphs(X_train_list.shape[0], symbols=symbol_names, hypervector_size=128, hypervector_bits=2)

# Step 2: Set the number of graph nodes for each training graph
for graph_id in range(X_train_list.shape[0]):
    graphs_train.set_number_of_graph_nodes(graph_id, dim * dim)

# Step 3: Prepare the node configuration
graphs_train.prepare_node_configuration()

# Step 4: Add nodes for each graph
for graph_id in range(X_train_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col  # Unique node ID for each hexagon

            # Dynamically calculate the number of neighbors for each node
            number_of_neighbors = 0
            if row > 0: number_of_neighbors += 1  # Above
            if row < dim - 1: number_of_neighbors += 1  # Below
            if col > 0: number_of_neighbors += 1  # Left
            if col < dim - 1: number_of_neighbors += 1  # Right
            if row > 0 and col < dim - 1: number_of_neighbors += 1  # AboveRight
            if row < dim - 1 and col > 0: number_of_neighbors += 1  # BelowLeft
            graphs_train.add_graph_node(graph_id, node_id, number_of_neighbors)



In [28]:
# Step 5: Add node properties for each graph
for graph_id in range(X_train_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col  # Unique node ID for each hexagon
            cell_value = X_train_list[graph_id, row, col]  # Get the cell's current state (Player 1, Player 2, or empty)

            # Feature 1: Value of each node (player occupancy)
            if cell_value == 1:
                graphs_train.add_graph_node_property(graph_id, node_id, "Blue")  # Player 1 occupies
            elif cell_value == -1:
                graphs_train.add_graph_node_property(graph_id, node_id, "Red")  # Player 2 occupies
            else:#cell_value == 0
                graphs_train.add_graph_node_property(graph_id, node_id, "White")     #empty
            

# Step 6: Prepare the edge configuration for the training data
graphs_train.prepare_edge_configuration()
# Step 7: Add edges between nodes (neighboring hexagons) for the training data
for graph_id in range(X_train_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col
            # Add edges to neighboring hexagons (hex grid has up to 6 neighbors per cell)
            if row > 0:  # Add edge to the hexagon above
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id - dim, "Above")
            if row < dim - 1:  # Add edge to the hexagon below
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id + dim, "Below")
            if col > 0:  # Add edge to the hexagon on the left
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id - 1, "Left")
            if col < dim - 1:  # Add edge to the hexagon on the right
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id + 1, "Right")
            if row > 0 and col < dim - 1:
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id - dim + 1, "AboveRight")
            if row < dim - 1 and col > 0:
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id + dim - 1, "BelowLeft")

print('finished')
graphs_train.encode()

print("Training data with new features produced")

finished
Training data with new features produced


In [29]:
graphs_test = Graphs(X_test_list.shape[0], symbols=symbol_names, hypervector_size=128, hypervector_bits=2)

# Step 2: Set the number of graph nodes for each training graph
for graph_id in range(X_test_list.shape[0]):
    graphs_test.set_number_of_graph_nodes(graph_id, dim * dim)

# Step 3: Prepare the node configuration
graphs_test.prepare_node_configuration()

# Step 4: Add nodes for each graph
for graph_id in range(X_test_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col  # Unique node ID for each hexagon

            # Dynamically calculate the number of neighbors for each node
            number_of_neighbors = 0
            if row > 0: number_of_neighbors += 1  # Above
            if row < dim - 1: number_of_neighbors += 1  # Below
            if col > 0: number_of_neighbors += 1  # Left
            if col < dim - 1: number_of_neighbors += 1  # Right
            if row > 0 and col < dim - 1: number_of_neighbors += 1  # AboveRight
            if row < dim - 1 and col > 0: number_of_neighbors += 1  # BelowLeft
            graphs_test.add_graph_node(graph_id, node_id, number_of_neighbors)

# Step 5: Add node properties for each graph
for graph_id in range(X_test_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col  # Unique node ID for each hexagon
            cell_value = X_test_list[graph_id, row, col]  # Get the cell's current state (Player 1, Player 2, or empty)
            
            # Feature 1: Value of each node (player occupancy)
            if cell_value == 1:
                graphs_test.add_graph_node_property(graph_id, node_id, "Red")  # Player 1 occupies
            elif cell_value == -1:
                graphs_test.add_graph_node_property(graph_id, node_id, "Blue")  # Player 2 occupies
            else:#cell_value == 0
                graphs_test.add_graph_node_property(graph_id, node_id, "White")     #empty

# Step 6: Prepare the edge configuration for the training data
graphs_test.prepare_edge_configuration()

# Step 7: Add edges between nodes (neighboring hexagons) for the training data
for graph_id in range(X_test_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col
            # Add edges to neighboring hexagons (hex grid has up to 6 neighbors per cell)
            if row > 0:  # Add edge to the hexagon above
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id - dim, "Above")

            if row < dim - 1:  # Add edge to the hexagon below
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id + dim, "Below")
            if col > 0:  # Add edge to the hexagon on the left
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id - 1, "Left")
            if col < dim - 1:  # Add edge to the hexagon on the right
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id + 1, "Right")
            if row > 0 and col < dim - 1:
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id - dim + 1, "AboveRight")

            if row < dim - 1 and col > 0:
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id + dim - 1, "BelowLeft")

# Step 8: After adding nodes and edges, encode the graph for the training data
graphs_test.encode()

print("Test data with new features produced_v_1")

Test data with new features produced_v_1


In [30]:
import argparse
import sys

def default_args(**kwargs):
    parser = argparse.ArgumentParser()
    parser.add_argument("--epochs", default=50, type=int)
    parser.add_argument("--number-of-clauses", default=300, type=int)
    parser.add_argument("--T", default=400, type=int)
    parser.add_argument("--s", default=1.2, type=float)
    parser.add_argument("--depth", default=3, type=int)
    parser.add_argument("--hypervector-size", default=512, type=int)
    parser.add_argument("--hypervector-bits", default=2, type=int)
    parser.add_argument("--message-size", default=512, type=int)
    parser.add_argument("--message-bits", default=2, type=int)
    parser.add_argument('--double-hashing', dest='double_hashing', default=False, action='store_true')
    parser.add_argument("--max-included-literals", default=16, type=int)

    # Add this line to avoid Jupyter arguments causing an error
    args = parser.parse_args(args=[])  # Ignore any command line arguments

    for key, value in kwargs.items():
        if key in args.__dict__:
            setattr(args, key, value)
    return args
args = default_args()

In [31]:

# Train the Tsetlin Machine
tm = MultiClassGraphTsetlinMachine(
    args.number_of_clauses,
    args.T,
    args.s,
    depth=args.depth,
    message_size=args.message_size,
    message_bits=args.message_bits,
    max_included_literals=args.max_included_literals,
    grid=(16*13,1,1),
    block=(128,1,1)
)

start_training = time()
for i in range(args.epochs):
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
    print(f"Epoch#{i+1} -- Accuracy train: {np.mean(Y_train == tm.predict(graphs_train))}", end=' ')
    print(f"-- Accuracy test: {np.mean(Y_test == tm.predict(graphs_test))} ")
stop_training = time()
print(f"Time: {stop_training - start_training}")

Initialization of sparse structure.
Epoch#1 -- Accuracy train: 0.9477777777777778 -- Accuracy test: 0.51 
Epoch#2 -- Accuracy train: 0.9861111111111112 -- Accuracy test: 0.655 
Epoch#3 -- Accuracy train: 0.9883333333333333 -- Accuracy test: 0.54 
Epoch#4 -- Accuracy train: 0.9894444444444445 -- Accuracy test: 0.655 
Epoch#5 -- Accuracy train: 0.9961111111111111 -- Accuracy test: 0.505 
Epoch#6 -- Accuracy train: 0.995 -- Accuracy test: 0.665 
Epoch#7 -- Accuracy train: 0.995 -- Accuracy test: 0.675 
Epoch#8 -- Accuracy train: 0.9961111111111111 -- Accuracy test: 0.61 
Epoch#9 -- Accuracy train: 0.995 -- Accuracy test: 0.54 
Epoch#10 -- Accuracy train: 0.9983333333333333 -- Accuracy test: 0.69 
Epoch#11 -- Accuracy train: 0.9983333333333333 -- Accuracy test: 0.675 
Epoch#12 -- Accuracy train: 1.0 -- Accuracy test: 0.59 
Epoch#13 -- Accuracy train: 0.9966666666666667 -- Accuracy test: 0.49 
Epoch#14 -- Accuracy train: 0.9827777777777778 -- Accuracy test: 0.505 
Epoch#15 -- Accuracy train

In [32]:
weights = tm.get_state()[1].reshape(2, -1)
for i in range(tm.number_of_clauses):
        print("Clause #%d W:(%d %d)" % (i, weights[0,i], weights[1,i]), end=' ')
        l = []
        for k in range(128 * 2):
            if tm.ta_action(0, i, k):
                if k < 128:
                    l.append("x%d" % (k))
                else:
                    l.append("NOT x%d" % (k - 128))
        print(" AND ".join(l))


Clause #0 W:(49 -26) x84 AND NOT x20 AND NOT x50 AND NOT x98 AND NOT x126
Clause #1 W:(75 -88) x123 AND NOT x19 AND NOT x23 AND NOT x24 AND NOT x29 AND NOT x35 AND NOT x49 AND NOT x57 AND NOT x72 AND NOT x73 AND NOT x77 AND NOT x85 AND NOT x95 AND NOT x107 AND NOT x122
Clause #2 W:(-5 -10) NOT x2 AND NOT x18 AND NOT x22 AND NOT x26 AND NOT x34 AND NOT x45 AND NOT x60 AND NOT x62 AND NOT x63 AND NOT x82 AND NOT x89 AND NOT x91 AND NOT x92 AND NOT x105 AND NOT x115 AND NOT x117 AND NOT x121 AND NOT x126
Clause #3 W:(-6 -16) NOT x0 AND NOT x7 AND NOT x8 AND NOT x15 AND NOT x20 AND NOT x25 AND NOT x33 AND NOT x34 AND NOT x37 AND NOT x57 AND NOT x79 AND NOT x100 AND NOT x105 AND NOT x108 AND NOT x111 AND NOT x113 AND NOT x124 AND NOT x127
Clause #4 W:(22 -24) NOT x0 AND NOT x6 AND NOT x40 AND NOT x44 AND NOT x54 AND NOT x56 AND NOT x57 AND NOT x65 AND NOT x72 AND NOT x75 AND NOT x76 AND NOT x83 AND NOT x87 AND NOT x91 AND NOT x107 AND NOT x116
Clause #5 W:(34 -56) NOT x2 AND NOT x4 AND NOT 